In [4]:
import numpy as np
import torch
import torch.nn as nn

**Key Notes**
- GP generalizes multivariate normal to _infinite_ dimension
- GP = _**distribution over functions**_
- GP is fully specified by a _**mean**_ function and _**covariance**_ function

In [5]:
print(torch.__version__)

0.4.0


In [6]:
import torch.nn as nn
import torch.nn.functional as F

In [9]:
!ls ../../..

Applications      Library           Untitled.ipynb    sdr-fmems.sdm
Desktop           Movies            fonts             workspace
Documents         Music             nltk_data
Downloads         Pictures          repos
Envs              Public            scikit_learn_data


In [10]:
class CNNClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, num_layers, num_labels, pre_trained_weights, dropout, bi_dir):
        super(GRUClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.bi_dir = 2 if bi_dir else 1
        
        self.num_layers = num_layers
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings.weight.data=torch.Tensor(pre_trained_weights)
        #self.rnn = nn.GRU(input_size = embedding_dim,
        ##                    hidden_size = hidden_dim,
        #                    num_layers = num_layers,
        #                    bidirectional = bi_dir
        #                   )
        self.drop = nn.Dropout(p=dropout)
        self.hidden2label = nn.Linear(hidden_dim*self.bi_dir, num_labels)
        self.hidden = self.init_hidden()

    def init_hidden(self, ):
        # the first is the hidden h
        return (Variable(torch.zeros(self.num_layers*self.bi_dir, 1,self.hidden_dim)))
        
    def forward(self, sentence):
        x = self.word_embeddings(sentence)
        x = self.drop(x)
        x = x.view(len(sentence), 1, -1)
        for i in range(self.num_layers):
            gru_out, self.hidden = self.rnn(x, self.hidden)
        out = self.hidden2label(gru_out[-1])
        log_probs = F.log_softmax(out)
        return log_probs
    
    def make_context_vector(self, seq, to_ix):
        idxs = [to_ix[w] for w in seq.split()]
        tensor = torch.LongTensor(idxs)
        return tensor
    
    def make_target(self, label, label_to_idx):
        return torch.LongTensor([label_to_idx[label]])
    
    def get_reply_proba(self, msg):
        x = transformText(msg,do_stop=False,do_stem=False)
        x = Variable(self.make_context_vector(x,word_to_ix))
        log_probs = self.forward(x)   
        _,predicted = torch.max(log_probs.data,1)
        pred_label=list(label_to_ix.keys())[list(label_to_ix.values()).index(predicted[0])]
        ## Getting probabilities
        probs = F.softmax(Variable(log_probs.data),dim=1)
        probs = probs.data[0].numpy()
        probs_frame = pd.DataFrame({'probs':probs,'label':list(label_to_ix)})
        probs_frame=probs_frame.sort_values(['probs'], ascending=False)[0:3].reset_index(drop=True)
        return pred_label,probs_frame